In [1]:
# May be needed
import os
os.chdir('../mangaki')

In [5]:
from mangaki.utils.data import Dataset

dataset = Dataset()
dataset.load('ratings-svd-20.pickle')

In [6]:
from scipy.sparse import csc_matrix
import numpy as np

matrix = csc_matrix((dataset.anonymized.y, (dataset.anonymized.X[:, 0], dataset.anonymized.X[:, 1])), dtype=np.float32)

In [7]:
import numpy as np
import tensorflow as tf

X = tf.cast(tf.SparseTensor(
    indices=dataset.anonymized.X,
    values=dataset.anonymized.y,
    dense_shape=(dataset.anonymized.nb_users, dataset.anonymized.nb_works),
), tf.float32)

# ALS

In [8]:
# For evaluation
indices = dataset.anonymized.X

In [9]:
N = dataset.anonymized.nb_users
R = 20
M = dataset.anonymized.nb_works

U = tf.Variable(np.random.rand(N, R).astype(np.float32), name='U')
V = tf.Variable(np.random.rand(R, M).astype(np.float32), name='V')

In [10]:
alpha = tf.constant(1e-4)
regu_U = alpha * tf.reduce_sum(tf.pow(U, 2))
regu_V = alpha * tf.reduce_sum(tf.pow(V, 2))

In [11]:
Y_est = tf.matmul(U, V)
true_loss = tf.reduce_sum(tf.pow(tf.gather_nd(Y_est, indices) - Y_tf.values, 2))
cost = true_loss + regu_U + regu_V

NameError: name 'Y_tf' is not defined

In [12]:
optimizer = tf.train.AdamOptimizer(0.005)
# train_step = optimizer.minimize(cost, var_list=[U, V])
train_U_step = optimizer.minimize(cost, var_list=[U])
train_V_step = optimizer.minimize(cost, var_list=[V])

NameError: name 'cost' is not defined

In [13]:
init_op = tf.global_variables_initializer()
sess.run(init_op)

NameError: name 'sess' is not defined

In [14]:
sess = tf.InteractiveSession()
sess.run(init_op)
for i in range(20):
    sess.run(train_U_step)
    sess.run(train_V_step)
    # sess.run(train_C_step)
    if i % 10 == 0:
        print(i, sess.run(cost), sess.run(true_loss))

NameError: name 'train_U_step' is not defined

- Batch by batch
- Try autoencoder with sigmoid then identity
- Maybe with SparseTensor?

In [15]:
#sess = tf.InteractiveSession()
#print(sess.run((tf.shape(X), tf.shape(V))))

N = dataset.anonymized.nb_users
R = 2
M = dataset.anonymized.nb_works

V = tf.Variable(tf.random_normal([M, R]), name='V')
mu = tf.Variable(tf.random_normal([R]), name='mu')

W = tf.Variable(tf.random_normal([R, M]), name='W')
b = tf.Variable(tf.random_normal([M]), name='b')

encoder = tf.nn.sigmoid(tf.add(tf.sparse_tensor_dense_matmul(X, V), mu))
y_pred = tf.add(tf.matmul(encoder, W), b)
Y_true = X
true_loss = tf.reduce_sum(tf.pow(tf.gather_nd(y_pred, indices) - Y_true.values, 2))

In [16]:
alpha = tf.constant(1e-2)
regu_V = alpha * tf.reduce_sum(tf.pow(V, 2))
regu_W = alpha * tf.reduce_sum(tf.pow(W, 2))
cost = true_loss + regu_V + regu_W

In [17]:
#optimizer = tf.train.AdamOptimizer(0.005)
#optimizer = tf.train.AdamOptimizer(1)
optimizer = tf.train.RMSPropOptimizer(0.1)
train_step = optimizer.minimize(cost, var_list=[V, mu, W, b])
#train_U_step = optimizer.minimize(cost, var_list=[U])
#train_V_step = optimizer.minimize(cost, var_list=[V])

In [18]:
init_op = tf.global_variables_initializer()

In [19]:
sess.run(init_op)
for i in range(20):
    #sess.run(train_U_step)
    #sess.run(train_V_step)
    _, c, t_l = sess.run([train_step, cost, true_loss])
    if i % 1 == 0:
        print(i, c, t_l)

0 1.40899e+06 1.40861e+06
1 956794.0 956440.0
2 853516.0 853168.0
3 789784.0 789442.0
4 743237.0 742899.0
5 708346.0 708012.0
6 679894.0 679562.0
7 657725.0 657396.0
8 639103.0 638776.0
9 623550.0 623224.0
10 609864.0 609540.0
11 598317.0 597994.0
12 588458.0 588136.0
13 579738.0 579417.0
14 572256.0 571936.0
15 565730.0 565410.0
16 560099.0 559781.0
17 555025.0 554707.0
18 550480.0 550163.0
19 546387.0 546069.0


In [148]:
sess.run(tf.gather_nd(y_pred, indices[:5]))

array([ 1.35544467,  0.60000342,  0.66582125,  1.2417649 ,  0.29592541], dtype=float32)

In [149]:
sess.run(Y_true.values[:5])

array([ 2. , -2. ,  0.1,  2. ,  2. ], dtype=float32)